In [1]:
#Background of character and situation

In [16]:
#Import Libraries
import pandas as pd
from pathlib import Path
import os
import hvplot.pandas
import numpy as np
from datetime import date
import seaborn as sns
import json
#import kaggle

In [17]:
#Import file from API (MUST HAVE KAGGLE INSTALLED: ("conda install kaggle")
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()


api.dataset_download_file('syuzai/perth-house-prices','all_perth_310121.csv')

ModuleNotFoundError: No module named 'kaggle'

In [18]:
#CSV file was unzipped from downloaded file

#Read CSV file
all_perth_data = pd.read_csv("all_perth_310121.csv")

In [19]:
#Drop NAs
all_perth_data = all_perth_data.dropna()

#Confirming all nulls have been removed
all_perth_data.isnull().sum()

ADDRESS             0
SUBURB              0
PRICE               0
BEDROOMS            0
BATHROOMS           0
GARAGE              0
LAND_AREA           0
FLOOR_AREA          0
BUILD_YEAR          0
CBD_DIST            0
NEAREST_STN         0
NEAREST_STN_DIST    0
DATE_SOLD           0
POSTCODE            0
LATITUDE            0
LONGITUDE           0
NEAREST_SCH         0
NEAREST_SCH_DIST    0
NEAREST_SCH_RANK    0
dtype: int64

In [20]:
#Remove \r from the Date_Sold column
all_perth_data['DATE_SOLD'] = all_perth_data['DATE_SOLD'].str.strip('\r')
all_perth_data['DATE_SOLD'] = pd.to_datetime(all_perth_data['DATE_SOLD'])

In [21]:
# Second option concatenation with another csv file
project_path_2 = Path("./residential_property_price.csv")

In [22]:
price_df = pd.read_csv(project_path_2)
price_df = price_df[price_df['MEDIAN HOUSE PRICE'] > 0]
price_df = price_df.set_index(price_df["SUBURB"])
price_df.head()

,SUBURB,MEDIAN HOUSE PRICE,GROWTH PERCENTAGE
SUBURB,,,
Alexander Heights,Alexander Heights,490000,8.90%
Alfred Cove,Alfred Cove,921000,17.00%
Alkimos,Alkimos,452500,10.40%
Applecross,Applecross,1735000,-0.60%
Ardross,Ardross,1165000,15.90%


In [23]:
suburbs_df = all_perth_data[['SUBURB', 'LATITUDE', 'LONGITUDE']].groupby(['SUBURB']).mean()
suburbs_df_price = pd.concat([price_df, suburbs_df], axis="columns", join="inner")
suburbs_df_price.head()

,SUBURB,MEDIAN HOUSE PRICE,GROWTH PERCENTAGE,LATITUDE,LONGITUDE
SUBURB,,,,,
Alexander Heights,Alexander Heights,490000,8.90%,-31.830310,115.866293
Alfred Cove,Alfred Cove,921000,17.00%,-32.033908,115.816117
Alkimos,Alkimos,452500,10.40%,-31.618988,115.691004
Applecross,Applecross,1735000,-0.60%,-32.013569,115.838399
Ardross,Ardross,1165000,15.90%,-32.026691,115.838054


In [24]:
#Turning Growth Percentage to Float data type to sort correctly
suburbs_df_price['GROWTH PERCENTAGE'] = suburbs_df_price['GROWTH PERCENTAGE'].str.strip('%')
suburbs_df_price['GROWTH PERCENTAGE'] = suburbs_df_price['GROWTH PERCENTAGE'].astype(float)

In [25]:
#Full Geo map grouped by suburbs
suburbs_df_price.hvplot.points('LONGITUDE',
                                       'LATITUDE', 
                                       geo=True,
                                       color='MEDIAN HOUSE PRICE',
                                       size='MEDIAN HOUSE PRICE',
                                       scale=0.013,
                                       tiles='OSM',
                                       frame_width=700,
                                       frame_height=500,
                                       hover = True,
                                       hover_cols=['SUBURB','MEDIAN HOUSE PRICE']
                                        )

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [LONGITUDE,LATITUDE]   (MEDIAN HOUSE PRICE,SUBURB)

Client 1 has come to us andbdgnwgjoekgfpsfk,ps

In [41]:
#Input of house parameters
df = all_perth_data.drop(columns=['BUILD_YEAR', 'POSTCODE', 'DATE_SOLD'])
bedrums = int(input('Please enter the number of Bedrooms : '))
plan_budget = int(input('Please enter your budget : '))
landsize = int(input('Please enter your desired minimum land size : '))
cbddist = int(input('Please enter the maximum distance from CBD in metres: '))
client1 = df.loc[(all_perth_data['BEDROOMS'] >= bedrums)
                               & (df['PRICE'] <= plan_budget)
                               & (df['CBD_DIST'] <= cbddist)
                               & (df['LAND_AREA'] >= landsize)]
client1

Please enter the number of Bedrooms :  5
Please enter your budget :  900000
Please enter your desired minimum land size :  800
Please enter the maximum distance from CBD in metres:  6000


,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK
15009,27 Ferrar Street,Menora,875000,5,2,1.0,835,200,3400,Mount Lawley Station,2200,-31.92216,115.86349,MOUNT LAWLEY SENIOR HIGH SCHOOL,0.600062,20.0
23095,46 Grosvenor Road,Bayswater,699000,5,1,2.0,859,157,5100,Meltham Station,543,-31.92658,115.90329,CHISHOLM CATHOLIC COLLEGE,1.695584,29.0
23590,49 Acton Avenue,Rivervale,750000,5,2,5.0,906,228,5400,Burswood Station,1600,-31.95611,115.91609,BELMONT CITY COLLEGE,1.758623,57.0
27574,60 Inverness Crescent,Menora,775000,5,3,4.0,822,55,3700,Perth Station,3600,-31.91858,115.85830,MOUNT LAWLEY SENIOR HIGH SCHOOL,1.120187,20.0


In [42]:
#Another map to show the options
client1.hvplot.points('LONGITUDE',
                                       'LATITUDE', 
                                       geo=True,
                                       color='ADDRESS',
                                       tiles='OSM',
                                       size= 'PRICE',
                                       scale= 0.05,
                                       frame_width=700,
                                       frame_height=500,
                                       hover = True,
                                       hover_cols=['SUBURB','PRICE', 'CBD_DIST', 'BEDROOMS']
                                        )

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [LONGITUDE,LATITUDE]   (ADDRESS,PRICE,SUBURB,CBD_DIST,BEDROOMS)

In [43]:
suburbs_df_price.loc[(suburbs_df_price["SUBURB"] == client1['SUBURB'])]

ValueError: Can only compare identically-labeled Series objects

In [ ]:
#Calculators
def mortgagecalculator (mortgage, years, interest):
    interest = interest / 100
    nper = years*12
    interest_monthly = interest/12
    numerator = interest_monthly*((1+interest_monthly)**nper)
    denominator = (1+interest_monthly)**nper-1
    payment = float("{0:.2f}".format(mortgage*numerator/denominator))
    return(payment)

In [ ]:
x=int(input('What is the mortgage amount? '))
y=int(input('For how many years? '))
z=int(input('What interest rate? '))
mortgagecalculator(x,y,z)

In [ ]:
def stampdutycalculator (purchase_price):
    if purchase_price <= 120000:
        stamp_duty = purchase_price*0.019
    elif purchase_price > 120000 and purchase_price <= 150000:
        stamp_duty = 2280 + (purchase_price - 120000) * 0.0285
    elif purchase_price > 150000 and purchase_price <= 360000:
        stamp_duty = 3135 + (purchase_price - 150000) * 0.038
    elif purchase_price > 360000 and purchase_price <= 725000:
        stamp_duty = 11175 + (purchase_price - 360000) * 0.0475
    else:
        stamp_duty = 28453 + (purchase_price - 725000) * 0.0515
    
    return stamp_duty

In [ ]:
q=int(input('What is the property value? '))
stampdutycalculator(q)

In [ ]:
#A new client story and parameters

In [ ]:
#New geo plot that contains house options for client B

In [ ]:
#Calculators

In [ ]:
#A couple of analysis questions including: If you had unlimited money, where would be the best value for money for an investment price per land area etc

#What are the top 3 suburbs to invest in by future growth percentage

#What are the top 3 suburbs to avoid by future growth percentage

In [34]:
suburbs_df_price = suburbs_df_price.sort_values(['GROWTH PERCENTAGE'], ascending=False)

In [35]:
display(suburbs_df_price.head(5), suburbs_df_price.tail(5))

,SUBURB,MEDIAN HOUSE PRICE,GROWTH PERCENTAGE,LATITUDE,LONGITUDE
SUBURB,,,,,
Gnangara,Gnangara,770000,94.6,-31.775330,115.848851
Hovea,Hovea,940000,82.5,-31.887604,116.113356
East Perth,East Perth,1950000,77.3,-31.952803,115.874626
Upper Swan,Upper Swan,790000,61.2,-31.769674,116.029006
Watermans Bay,Watermans Bay,1670000,51.8,-31.849449,115.755512


,SUBURB,MEDIAN HOUSE PRICE,GROWTH PERCENTAGE,LATITUDE,LONGITUDE
SUBURB,,,,,
Maylands,Maylands,655000,-5.1,-31.934201,115.900674
Sawyers Valley,Sawyers Valley,565000,-5.8,-31.893090,116.220120
Herne Hill,Herne Hill,370000,-11.9,-31.829031,116.027816
Darling Downs,Darling Downs,530000,-13.3,-32.199243,115.992782
Carmel,Carmel,555000,-14.6,-32.014680,116.072370
